In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pylhef
import sys
%matplotlib notebook

lhfile = pylhef.read('test_events.lhe')

In [ ]:
sim_events = []

for event in lhfile.events:
    part = event.particles[2]
    pid = part.id
    inout = 1
    fmom = part.first_mother
    lmom = part.last_mother
    col1 = part.color[0]
    col2 = part.color[1]
    px = part.p[1]
    py = part.p[2]
    pz = part.p[3]
    energy = part.p[0]
    mas = part.mass
    lt = part.lifetime
    spi = part.spin
    
    simpart = [pid,inout,fmom,lmom,col1,col2,px,py,pz,energy,mas,lt,spi]
    sim_events.append(simpart)

In [ ]:
test=open('test_events.lhe','r')
file=[]

for line in test:
    file.append(line)
    
lines=[i for i,x in enumerate(file) if x=='<event>\n']
len(lines) # = 94

sims=[]
n=0
while n<=len(lines):
    if n==0:
        place = lines[n]+5
        sims.append(file[0:place])
        sims.append(sim_events[n])
    elif n < len(lines):
        place = lines[n]+5
        last = lines[n-1]+5
        sims.append(file[last:place])
        sims.append(sim_events[n])
    else:
        last = lines[n-1]+5
        sims.append(file[last:])
    n=n+1